# Deep learning framework example: Named Entity Recognition

This notebook demonstrates how to use the deeplearning API to train and test the model on the [CoNNL 2003 dataset](https://www.clips.uantwerpen.be/conll2003/ner/).  The task is to identify named entities (i.e. people, organizations etc).

**Important**: Please see the Movie Review notebook example in the `zensols.movie` API first, as it contains more explaination of how the framework is used.  The purpose of this notebook is to run the MNIST dataset and visualize the results.

In [1]:
# set up notebook environment
import sys
from pathlib import Path
app_root_dir = Path('..')
# add the example to the Python library path
sys.path.append(str(app_root_dir / 'src'))
# add the deepnlp path
sys.path.append('../../../src/python')

## Configure and create the app specific facade

Now that the interpreter environment is set up, we can import local packages.

In [2]:
from zensols.deeplearn import TorchConfig
from zensols.deeplearn.cli import JupyterManager
from ner import CliFactory

# reset random state for consistency of each new test that uses this function
TorchConfig.init()

mng = JupyterManager(
    allocation_tracking=True,
    cli_class=CliFactory,
    factory_args={'root_dir': app_root_dir},
    cli_args_fn=lambda model: ['-c', str(app_root_dir / 'models' / f'{model}.conf')])

## Print information about 

Use the factory to create the model executor.  The `write` method gives statistics on the data set that is configured on the executor.

In [3]:
from zensols.config import Writable
# set indention level for human readable (pretty print) output
Writable.WRITABLE_INDENT_SPACE = 2
facade = mng.create_facade('glove50')
facade.write()

2021-06-17 14:42:25,173 [zensols.deeplearn.model.facade] creating new executor
2021-06-17 14:42:26,060 [zensols.deepnlp.embed.wordtext] reading binary vector file: ../corpus/glove/bin/6B.50/vec
2021-06-17 14:42:26,368 [zensols.deepnlp.embed.wordtext] loaded 400000 vectors in 0s
2021-06-17 14:42:26,650 [zensols.deepnlp.embed.wordtext] prepared vectors in 0s
2021-06-17 14:42:26,651 [zensols.deepnlp.embed.domain] created tensor vectory matrix on use cuda: True, device: cuda:0


executor:
  model: NER: glove_50 
  feature splits:
    split stash splits:
        test: 3453 (16.6%)
        train: 14041 (67.7%)
        dev: 3250 (15.7%)
        total: 20744


2021-06-17 14:42:29,930 [zensols.deeplearn.model.executor.status] created model on cpu with use cuda: True, device: cuda:0


    total this instance: 20744
    keys consistent: True
    delegate:
      key splits:
        test: 3453 (16.6%)
        train: 14041 (67.7%)
        dev: 3250 (15.7%)
      total: 20744
  batch splits:
    split stash splits:
        test: 3 (33.3%)
        train: 3 (33.3%)
        dev: 3 (33.3%)
        total: 9
    total this instance: 9
    keys consistent: True
    delegate:
      name: sent_batch_stash
      chunk_size: 0
      workers: 2
      batch_size: 4
      data_point_id_sets_path: ../data/batch/b100/keys.dat
      batch_limit: 3
      delegate_attr: True
      _has_data: True
      is_child: False
      _decoded_attributes: {'mask', 'tags', 'glove_50_embedding', 'ents', 'syns'}
      priming: False
  network settings:
    name: net_settings
    recurrent_crf_settings:
      name: recurrent_crf_settings
      dropout: 0.1
      network_type: lstm
      bidirectional: True
      hidden_size: 24
      num_layers: 1
      num_labels: 9
      decoder_settings:
        name:

## Train and test the model

Train and test the model with the default (low) number of epochs to make sure everything is working.

In [4]:
facade.epochs = 2
mng.run()

2021-06-17 14:42:29,943 [zensols.deeplearn.model.executor.status] resetting executor
2021-06-17 14:42:29,955 [zensols.deeplearn.model.facade] training...
2021-06-17 14:42:29,964 [zensols.deeplearn.model.executor.status] batch iteration: gpu, limit: 9223372036854775807, caching: True, cached: 0
2021-06-17 14:42:29,964 [zensols.deeplearn.model.executor.status] preparing datasets using iteration: gpu
2021-06-17 14:42:29,965 [zensols.deeplearn.model.executor.status] using batch limit: 9223372036854775807 for train
2021-06-17 14:42:29,982 [zensols.deeplearn.model.executor.status] using batch limit: 9223372036854775807 for dev
2021-06-17 14:42:29,995 [zensols.deeplearn.model.executor.status] loaded 6 batches in 0s
2021-06-17 14:42:29,995 [zensols.deeplearn.model.executor.status] train/test sets: 3 3
2021-06-17 14:42:30,001 [zensols.deeplearn.model.executor.status] created model on cpu with use cuda: True, device: cuda:0
2021-06-17 14:42:30,031 [zensols.deeplearn.model.executor.status] traini

LayerError: The first two dimensions of emissions and tags must match, got (4, 15) and (4, 100)

## Tune hyperparameters

Set model parameters to get a feel for where they need to be before changing features.  Start with Glove 50 dimensional word embeddings with a learning rate of 0.01 and 20 epochs.

In [ ]:
facade.learning_rate = 0.01
facade.epochs = 20
mng.run()

# Glove 300 embeddings

Next we use the same learning rate, but switch to the 300 dimension version of the embeddings.  The number of epochs is reduced because I have run the test before I know at what epoch the validation loss converges.  Since the model is saved only when the validation loss decreases, we early stop at 8 epochs.

In [ ]:
facade.epochs = 8
facade.embedding = 'glove_300_embedding'
mng.run()

# Word2vec Embeddings

Now we switch to the Google 300D word2vec pretrained vectors using 12 epochs, even though it has converged at 9 epochs previously.

In [ ]:
facade.epochs = 12
facade.embedding = 'word2vec_300_embedding'
mng.run()

# BERT Embeddings

Now we test with Bert context aware frozen (not trainable) embeddings using 10 epochs.  We must empty the `net_settings` attributes, which are the lingustic features, since Bert tokenizes using the word piece algorithm and the tensor shapes will not align.  We'll address this later.

In [ ]:
facade = mng.create_facade('transformer-fixed')
facade.epochs = 10
facade.net_settings.add_attributes = ()
mng.run()

In [ ]:
facade.net_settings.add_attributes = ('syns_expander', 'tags_expander')
mng.run()

In [ ]:
facade = mng.create_facade('transformer-trainable')
mng.run()